In [8]:
import os
import openai

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
    ServiceContext,
    Document
)
from llama_index.llms.openai import OpenAI
# from llama_index.llms.anthropic import Anthropic

from llama_index.core.node_parser import SentenceWindowNodeParser, HierarchicalNodeParser, get_leaf_nodes
from llama_index.core.text_splitter import SentenceSplitter
#from llama_index.core.embeddings import OpenAIEmbedding, HuggingFaceEmbedding
from llama_index.core.schema import MetadataMode
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

In [10]:
openai_key = os.getenv("OPENAI_API_KEY")

In [11]:
documents = SimpleDirectoryReader("./data/").load_data()

In [15]:
# Inspect the documents
print("length of doc: "+ str(len(documents)))
print("----")
print(documents)

length of doc: 80
----
[Document(id_='59c4edfe-47a9-488e-86a6-db807c887678', embedding=None, metadata={'page_label': '1', 'file_name': 'Automated External Defibrillator (1).pdf', 'file_path': '/Users/shakthiraveen/Desktop/AI-Driven.nosync/RAG-new/data/Automated External Defibrillator (1).pdf', 'file_type': 'application/pdf', 'file_size': 574267, 'creation_date': '2024-04-10', 'last_modified_date': '2024-04-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="Automated External Defibrillator\nHome  EHS Guidelines  Downloads  Case Studies  Webinars  Contact Us  Search\nAutomated External Defibrillator:\nOverview:\nTo understand how AEDs work, it helps to understand .\nThe heart has an internal electrical system that controls the rate and rhythm of the hea

In [16]:
documents[0].metadata
documents[1].metadata

{'page_label': '2',
 'file_name': 'Automated External Defibrillator (1).pdf',
 'file_path': '/Users/shakthiraveen/Desktop/AI-Driven.nosync/RAG-new/data/Automated External Defibrillator (1).pdf',
 'file_type': 'application/pdf',
 'file_size': 574267,
 'creation_date': '2024-04-10',
 'last_modified_date': '2024-04-10'}

In [17]:
# create the sentence window node parser w/ default settings
sentence_node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text"
)

base_node_parser = SentenceSplitter()

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [18]:
nodes = sentence_node_parser.get_nodes_from_documents(documents)
base_nodes = base_node_parser.get_nodes_from_documents(documents)

In [22]:
print("---------")
print("SENTENCE NODES")
print("---------")
print(nodes[100])
print("---------")
print("BASE NODES")
print("---------")
print(base_nodes[50])

---------
SENTENCE NODES
---------
Node ID: ae80fea8-6323-4ca9-bb3b-750fb2ffe559
Text: After 2 minutes of CPR, you can use the AED again to check the
person's heart rhythm and give another shock, if needed.
---------
BASE NODES
---------
Node ID: a33802fb-eb20-463e-af0c-7aab66dca31d
Text: First aid - Heat rash Home  EHS Guidelines  Downloads  Case
Studies  Webinars  Contact Us  Search First aid - Heat rash:
Definitions: Heat rash — also known as prickly heat and miliaria —
isn't just for babies. Heat rash affects adults, too, especially
during hot, humid weather. Heat rash develops when blocked pores
(sweat ducts) trap perspirati...


In [24]:
dict(base_nodes[50])

{'id_': 'a33802fb-eb20-463e-af0c-7aab66dca31d',
 'embedding': None,
 'metadata': {'page_label': '1',
  'file_name': 'First aid - Heat rash.pdf',
  'file_path': '/Users/shakthiraveen/Desktop/AI-Driven.nosync/RAG-new/data/First aid - Heat rash.pdf',
  'file_type': 'application/pdf',
  'file_size': 593124,
  'creation_date': '2024-04-10',
  'last_modified_date': '2024-04-10'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='8d8cf593-a77e-4f8e-9083-ac51f881b7be', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'First aid - Heat rash.pdf', 'file_path': '/Users/shakthiraveen/Desktop/AI-Driven.nosync/RAG-new/data/First aid - Heat rash.pdf', 'file_type

In [26]:
from llama_index.embeddings.openai import OpenAIEmbedding

ctx_sentence = ServiceContext.from_defaults(llm=llm, embed_model=OpenAIEmbedding(embed_batch_size=50), node_parser=sentence_node_parser)
ctx_base = ServiceContext.from_defaults(llm=llm, embed_model=OpenAIEmbedding(embed_batch_size=50), node_parser=base_node_parser)

sentence_index = VectorStoreIndex(nodes, service_context=ctx_sentence)
base_index = VectorStoreIndex(base_nodes, service_context=ctx_base)

/var/folders/by/93n0jb0x7h1dp51_sqxqqfg80000gn/T/ipykernel_7702/2402327980.py:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  ctx_sentence = ServiceContext.from_defaults(llm=llm, embed_model=OpenAIEmbedding(embed_batch_size=50), node_parser=sentence_node_parser)
/var/folders/by/93n0jb0x7h1dp51_sqxqqfg80000gn/T/ipykernel_7702/2402327980.py:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  ctx_base = ServiceContext.from_defaults(llm=llm, embed_model=OpenAIEmbedding(embed_batch_size=50), node_parser=base_node_parser)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: PO

In [27]:
sentence_index.storage_context.persist(persist_dir="./sentence_index")
base_index.storage_context.persist(persist_dir="./base_index")

In [28]:
# rebuild storage context
SC_retrieved_sentence = StorageContext.from_defaults(persist_dir="./sentence_index")
SC_retrieved_base = StorageContext.from_defaults(persist_dir="./base_index")

In [29]:
# load index
retrieved_sentence_index = load_index_from_storage(SC_retrieved_sentence)
retrieved_base_index = load_index_from_storage(SC_retrieved_base)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [38]:
from llama_index.core.postprocessor import SimilarityPostprocessor

sentence_query_engine = retrieved_sentence_index.as_query_engine(
    similarity_top_k=3,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

base_query_engine = retrieved_base_index.as_query_engine(
    similarity_top_k=5,
)

In [42]:
question = "i cut my finger while cooking what should i do? give the answer in steps"

In [43]:
base_response = base_query_engine.query(
    question
)
print(base_response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 

In [44]:
sentence_response = sentence_query_engine.query(
    question
)
print(sentence_response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 